<a href="https://colab.research.google.com/github/zephyris/tryptag/blob/main/examples/open_cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install tryptag module

#@markdown Install the `tryptag` module using `pip`.

!pip install git+https://github.com/zephyris/tryptag

In [ ]:
#@title Setup tryptag

#@markdown The `tryptag` module handles all data retrieval.

# import and set up TryTag instance
from tryptag import (
    TrypTag,
    CellLine,
    CellImage,
    GeneNotFoundError,
    InvalidTerminusError,
    FieldNotFoundError,
    CellNotFoundError,
)
tryptag = TrypTag()

# import skimage and numpy, for handling and displaying the images
import skimage
import numpy
from PIL import Image

# filter matplotlib warnings, used by skimage for image display
import warnings
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

# function for quickly displaying some images
def quickshow_images(image):
    # show composite
    display(image)
    # show individual channels
    print("phase")
    display(image.phase)
    print("mng")
    display(image.mng)
    print("dna")
    display(image.dna)
    print("phase_mask")
    display(image.phase_mask)
    print("dna_mask")
    display(image.dna_mask)

In [ ]:
#@title Open images for a cell

#@markdown Cells are opened using their `gene_id` (from [TriTrypDB](https://tritrypdb.org)) and tagged `terminus`, which must be `"n"` or `"c"`.
gene_id = "Tb927.9.8570" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
#@markdown Most cell lines have multiple fields of view and many cells per field of view. You can choose these using `field_index` and `cell_index`.
field_index = 2 #@param {type:"integer"}
cell_index = 8 #@param {type:"integer"}

#@markdown The `tryptag.open_cell` returns a `CellImage` object. This includes 6 `skimage` images:
#@markdown 0. `.phase` Phase contrast
#@markdown 1. `.mng` mNG (tagged protein) fluorescence
#@markdown 2. `.dna` Hoechst 33342 (DNA stain) fluorescence
#@markdown 3. `.phase_mask` Mask from thresholded phase contrast for the current cell
#@markdown 4. `.dna_mask` Mask from thresholded DNA stain
#@markdown 5. `.phase_mask_othercells` Mask from thresholded phase contrast for other cells

#@markdown The first time you open a cell it will take some time for `tryptag` to download the necessary data from the repository.
#@markdown The data is saved in a cache directory, so the next time you open any cell for this cell line it will be far quicker.
#@markdown Data is in 96 well plate groupings and download time will depend on how many images were captured for that plate - all this data will be cached.
#@markdown In Google Colab this cache will (unfortunately) be deleted at the end of each session, but on your computer would be stored permanently.

# create a CellLine object to pass the data
cell_line = CellLine(gene_id=gene_id, terminus=terminus)

# open the cell
cell_image = tryptag.open_cell(cell_line, field_index, cell_index)

# show the images
quickshow_images(cell_image)

In [ ]:
#@title Open cell image options

#@markdown The default behaviour when opening a cell image is to give a square image 323 pixels wide cropped around the centre of the cell.
#@markdown This can be customised:

#@markdown Provide a positive `width` parameter to crop to a square of that width.
positive_width = 120 #@param {type:"integer"}
cell_image = tryptag.open_cell(cell_line, field_index, cell_index, width = positive_width)
print("width", positive_width)
display(cell_image.phase_mask)
print("")

#@markdown Set `rotate` to `True` to provide a nice rotated image (with half height) - good for figures, but not so good for image analysis!
#@markdown You can also set a custom `angle` in degrees, or not set an angle to use the `tryptag` default.
rotate = True #@param {type:"boolean"}
cell_image = tryptag.open_cell(cell_line, field_index, cell_index, rotate = rotate)
print("rotate", rotate)
display(cell_image.phase_mask)
print("")

#@markdown Provide a negative `width` parameter to crop to the `phase_mask` of the cell, but with a border of `-width` pixels.
#@markdown ie. `width = -32` will give a 32 pixel border around the mask from the phase image.
negative_width = -32 #@param {type:"integer"}
cell_image = tryptag.open_cell(cell_line, field_index, cell_index, width = negative_width)
print("width", negative_width)
display(cell_image.phase_mask)

#@markdown The images will always be of the requested width / have the requested border.
#@markdown If necessary, the edge of the image will be extended with the median of the field of view.

In [ ]:
#@title Image processing

#@markdown You can access the different images in the returned `CellImage` object using the appropriate dot notation.
#@markdown For example, `cell_images.phase`.

#@markdown The opened `skimage` images can be processed, analysed and/or saved using `skimage`.
#@markdown For example using `skimage.restoration.rolling_ball` to generate a background estimate which can be subtracted.

#@markdown `skimage` images are `numpy` 2D `ndarray` objects, and can also be handled using `numpy` tools.
#@markdown This includes simple mathematical operations like addition, subtraction, multiplication, etc.
#@markdown For example, using `numpy` to measure median background, then subtract this from the image.

# mng
print("mNG fluorescence")
display(cell_image.mng)
print("")

# mng, minus median
print("... minus the image median and clipped to positive values")
mng_bgs = cell_image.mng - numpy.median(cell_image.mng)
mng_bgs[mng_bgs < 0] = 0
display(mng_bgs)
print("")

# mng, with rolling ball filter
print("... with rolling ball filter, radius 5 px")
mng_rb = skimage.restoration.rolling_ball(cell_image.mng, radius = 5)
display(cell_image.mng - mng_rb)
print("")

# mng, with rolling ball filter
print("... median subtracted and masked with phase contrast mask image")
mng_ma = (cell_image.mng - numpy.median(cell_image.mng)) * cell_image.phase_mask / 255
display(mng_ma)

In [ ]:
#@title Open field of view

#@markdown Fields of view can also be opened from their `gene_id`, tagged `terminus`, and `field_index`.
gene_id = "Tb927.9.8570" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
field_index = 1 #@param {type:"integer"}

# create a CellLine object to pass the data
cell_line = CellLine(gene_id=gene_id, terminus=terminus)

#@markdown Much like `tryptag.open_cell`, `tryptag.open_field` returns a `FieldImage` object with all except the `.phase_mask_othercells` image.

# open the field
field_image = tryptag.open_field(cell_line, field_index)

# show the images
quickshow_images(field_image)

In [ ]:
#@title Advanced usage

from tryptag import FieldImage

#@markdown Custom field of view images can be injected into the opening of cell images using `open_cell_custom`.
#@markdown This is particularly useful if doing computationally costly image modifications prior to analysing all of the cells in a field of view.

#@markdown Custom/altered images are provided as a `FieldImage` object, often easiest to modify the object returned by `open_field`.
#@markdown Any can be either an `skimage` image or `None`, if `None` then the default image will not be replaced.

gene_id = "Tb927.9.8570" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
field_index = 2 #@param {type:"integer"}
cell_index = 8 #@param {type:"integer"}

# create a CellLine object to pass the data
cell_line = CellLine(gene_id=gene_id, terminus=terminus)

#@markdown Select which changes to make:
invert_phase = True #@param {type:"boolean"}
rolling_ball_mng = True #@param {type:"boolean"}
median_subtract_dna = True #@param {type:"boolean"}
size_filter_dna_mask = True #@param {type:"boolean"}
#@markdown Any alterations to the thresholded phase contrast image might change where to find cells.

# open field of view
field_image = tryptag.open_field(cell_line, field_index)

# do modifications:
# invert phase contrast image
phase = None
if invert_phase:
    phase = int(2**16-1) - field_image.phase

# rolling ball subtraction for mng
mng = None
if rolling_ball_mng:
    mng = field_image.mng - skimage.restoration.rolling_ball(field_image.mng, radius = 5)

# median subtraction for dna
dna = None
if median_subtract_dna:
    dna = field_image.dna - numpy.median(field_image.dna)

# size filter dth
dna_mask = None
if size_filter_dna_mask:
    dna_labelled = skimage.measure.label(field_image.dna_mask)
    dna_mask = skimage.morphology.remove_small_objects(dna_labelled, min_size = 250)

# make a new FieldImage object to pass to the open_cell_custom function
new_field_image = FieldImage(
    phase = phase,
    mng = mng,
    dna = dna,
    phase_mask = None,
    dna_mask = dna_mask,
    cell_line = cell_line,
    field_index = field_image.field_index,
)

# open cell images, using modified field images
cell_images = tryptag.open_cell(cell_line, field_index, cell_index, custom_field_image = new_field_image)

#@markdown More advanced analysis can alter or generate a new thresholded phase contrast image.
#@markdown This, along with custom `fill_centre`, `crop_centre` and `angle` parameters for cell locations, can be passed to `open_cell_custom`.

quickshow_images(cell_images)

In [ ]:
#@title Handling errors

#@markdown If you try to load a `gene_id`, `terminus`, `field_index` or `cell_index` which does not exist then you'll get an error.

bad_gene_id = "Tb927.12.8570" #@param {type:"string"}
bad_terminus = "o" #@param ["n", "c"]
bad_field_index = 15 #15 #@param {type:"integer"}
bad_cell_index = 120 #@param {type:"integer"}

#@markdown Make sure you:
#@markdown 1. either use the `try` `except` syntax,

print("Try except method:")
try:
    # create a CellLine object to pass the data
    bad_cell_line = CellLine(gene_id=bad_gene_id, terminus=bad_terminus)
    cell_image = tryptag.open_cell(bad_cell_line, bad_field_index, bad_cell_index)
except GeneNotFoundError as e:
    print(f"Oops, your Gene ID {e.geneid} was not found!")
except InvalidTerminusError as e:
    print(f"Oops, the terminus {e.terminus} is invalid!")
except FieldNotFoundError as e:
    print(f"Oops, the field with index {e.index} could not be found!")
except CellNotFoundError as e:
    print(f"Oops, the cell with index {e.index} could not be found!")
print("")

#@markdown 2. or check that the necessary values in the `tryptag.gene_list` object exist.

#@markdown The latter is powerful, but not user friendly.
#@markdown If checking against `tryptag.gene_list` then make sure to request a list of cells using `cell_list` first.
#@markdown This triggers data download then counting of fields of view and cells once the data is in the cache.

print("Manual gene_list check method:")
if bad_gene_id in tryptag.gene_list:
    gene = tryptag.gene_list[bad_gene_id]
    if bad_terminus in tryptag.gene_list[bad_gene_id]:
        cell_line = gene[bad_terminus]
        # fetch the data for this gene_id and terminus
        if bad_field_index in cell_line.fields:
            field = cell_line.fields[bad_field_index]
            if bad_cell_index in field.cells:
                cell = field.cells[bad_cell_index]
                cell_image = CellImage(cell, rotated=False)
            else:
                print("cell_index", bad_cell_index, "out of range")
        else:
            print("field_index", bad_field_index, "out of range")
    else:
        print("data for terminus", bad_terminus, "not found")
else:
  print("data for gene_id", bad_gene_id, "not found")